In [100]:
# Imports
import pandas as pd
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from statistics import mean, median
from scipy.stats import skewnorm, gamma
from tqdm import tqdm
import random
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
#from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from statistics import mean, median
from tensorflow.keras import models, layers
from tensorflow.keras.layers import BatchNormalization, Activation, Conv1D, Input, Flatten, Dense, Dropout, MaxPool1D
from tensorflow.keras.optimizers import schedules, SGD, Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.initializers import HeNormal
import tensorflow as tf
import keras
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
from keras import regularizers, losses, metrics

In [4]:
# Load the dataset
df = pd.read_csv('train_dataset_RAW.csv')

In [140]:
# Parse the dataset
c = df['C']
si = df['Si']
mn = df['Mn']
cr = df['Cr']
mo = df['Mo']
ni = df['Ni']
cu = df['Cu']
fe = df['Fe']
clean_df=df.T
clean_df.drop(clean_df.tail(8).index, inplace=True)
clean_df.index = clean_df.index.to_series().astype(str).str.replace('X','',regex=True).astype(float)
clean_df.index = clean_df.index.astype(float)
clean_df = clean_df.loc[357.75:358,:]
clean_df.shape

(13, 2100)

In [141]:
# Divide data into training/validation and testing set and reshape
unique_test_id = [1, 6, 14, 24, 26, 30, 39]
test_id = [list(range(i*50,i*50+50)) for i in unique_test_id]
test_id = [item for sublist in test_id for item in sublist]
x_train_val = clean_df.drop(test_id, axis=1).to_numpy()
y_train_val = cr.drop(test_id).to_numpy()
x_test = clean_df.iloc[:,test_id].to_numpy()
y_test = cr.iloc[test_id].to_numpy()
x_train_val = np.swapaxes(x_train_val, 0, 1)
x_test = np.swapaxes(x_test, 0, 1)

# Scale data
x_scaler_mean = np.mean(x_train_val)
x_scaler_std = np.std(x_train_val)
X_train_val = (x_train_val-x_scaler_mean)/x_scaler_std
X_test = (x_test-x_scaler_mean)/x_scaler_std

In [124]:
ica = FastICA(n_components=50, max_iter = 1000)
train_val_features = ica.fit_transform(X_train_val)
test_features = ica.transform(X_test)

C:\Users\tdvorak\Desktop\bakalarka_lul\.venv\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


In [123]:
train_val_features.shape

(1750, 20)

In [94]:
test_features.shape

(350, 100)

In [148]:
#Define model architecture
model = models.Sequential()
model.add(layers.Dense(8, activation='LeakyReLU',input_shape = (13 ,1)))
#model.add(layers.Dropout(0.1))
model.add(layers.Dense(8, activation='LeakyReLU'))
#model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation="linear"))
model.compile(loss='mse', optimizer=SGD(learning_rate=0.005), metrics=['mae'])
model.summary()

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_120 (Dense)           (None, 13, 8)             16        
                                                                 
 dense_121 (Dense)           (None, 13, 8)             72        
                                                                 
 dense_122 (Dense)           (None, 13, 1)             9         
                                                                 
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


In [149]:
history = model.fit(X_train_val, y_train_val, epochs=100, batch_size=1)

Epoch 1/100
1750/1750 [==============================] - 3s 1ms/step - loss: 108.5600 - mae: 9.0973
Epoch 2/100
1750/1750 [==============================] - 2s 1ms/step - loss: 102.1455 - mae: 8.9582
Epoch 3/100
1750/1750 [==============================] - 2s 1ms/step - loss: 101.3885 - mae: 8.9296
Epoch 4/100
1750/1750 [==============================] - 2s 1ms/step - loss: 101.4909 - mae: 8.9025
Epoch 5/100
1750/1750 [==============================] - 2s 1ms/step - loss: 101.4232 - mae: 8.8942
Epoch 6/100
1750/1750 [==============================] - 2s 1ms/step - loss: 101.3333 - mae: 8.9069
Epoch 7/100
1750/1750 [==============================] - 2s 1ms/step - loss: 101.2395 - mae: 8.8940
Epoch 8/100
1750/1750 [==============================] - 2s 1ms/step - loss: 101.2802 - mae: 8.9021
Epoch 9/100
1750/1750 [==============================] - 3s 1ms/step - loss: 100.9494 - mae: 8.8586
Epoch 10/100
1750/1750 [==============================] - 2s 1ms/step - loss: 101.5172 - mae: 8.9073

KeyboardInterrupt: 

10.303537142857143

In [ ]:
# Model architecture
def buildmodel():
    model = models.Sequential()
    model.add(layers.Conv1D(3, 3, activation='relu', input_shape = (40002 ,1)))
    model.add(layers.MaxPooling1D(pool_size = 2))
    model.add(layers.Conv1D(5, 5, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size = 2))
    model.add(layers.Conv1D(7, 7, activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="linear"))



    model.compile(loss='mse', optimizer=Adam(learning_rate=0.002), metrics=['mae'])
    return model
    
model = buildmodel()
model.summary()

In [ ]:
# Train model on all train_val data
history = model.fit(X_train_val, y_train_val, epochs=100, batch_size=1, verbose=1)

In [ ]:
# Create a class activation map
conv_output = model.get_layer('conv1d_23').output

model2 = models.Model(model.inputs, outputs=[model.layers[1].output, model.layers[-1].output])

conv, pred = model2.predict(X_train_val[np.newaxis,1])

w, b = model2.get_layer("conv1d_23").weights
target = np.argmax(pred, axis=1).squeeze()
weights = w[:, target].numpy()
heatmap = conv.squeeze() @ weights

In [ ]:
a = model2.predict(X_train_val[np.newaxis,1])

In [ ]:
model.layers[0].output